In [ ]:
# ==============================================================================
# CÉLULA ÚNICA FINAL PARA DataAnalysis2.ipynb
# ==============================================================================

import pandas as pd
from pathlib import Path
import sys
import matplotlib.pyplot as plt

# --- 1. SETUP E INICIALIZAÇÃO ---
print("--- 1. INICIANDO SETUP ---")
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

import DataUtils as du
from DataUtils import AnalisadorBancario

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100) # Para ver mais nomes de bancos
plt.style.use('seaborn-v0_8-whitegrid')

output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))
print("-" * 50)

In [ ]:
# --- 3. TESTE DE CONSULTA BÁSICA (COSIF) ---
print("\n--- 3. TESTANDO CONSULTA BÁSICA COSIF ---")
# Use um nome exato da lista acima para garantir o sucesso
dados_bb_cosif = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=['ATIVO REALIZÁVEL', 'Caixa'],
    datas=202403
)
print("Resultado para 'BCO DO BRASIL S.A.':")
display(dados_bb_cosif)
print("-" * 50)


# --- 4. TESTE DA TABELA COMPARATIVA COMPLETA ---
print("\n--- 4. TESTANDO TABELA COMPARATIVA COMPLETA ---")
indicadores_completos = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMÔNIO LÍQUIDO'},
    'Total Ativos': {'tipo': 'COSIF', 'conta': 'TOTAL GERAL DO ATIVO'},
    'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
    'Situação': {'tipo': 'Atributo', 'atributo': 'SITUACAO_IFD_CAD'}
}

# Usando nomes curtos, mas agora sabendo que a busca flexível funciona
bancos_para_comparar = ['61190658', 'BANCO BRADESCO', 'NU FINANCEIRA', 'BANCO BTG PACTUAL', 'BCO DO BRASIL S.A.']

print("\nComparativo usando Documento 4060 (Balancete):")
tabela_balancete = analisador.comparar_indicadores(
    identificadores=bancos_para_comparar,
    indicadores=indicadores_completos,
    data=202403,
    documento_cosif=4060 
)
display(tabela_balancete)
print("-" * 50)


# --- 5. TESTE DA SÉRIE TEMPORAL ---
print("\n--- 5. TESTANDO SÉRIE TEMPORAL ---")
serie_nubank = analisador.get_serie_temporal_indicador(
    identificador='NU FINANCEIRA', # Usando nome curto
    conta='TOTAL GERAL DO ATIVO',
    data_inicio=202401,
    data_fim=202412,
    fonte='COSIF',
    documento_cosif=4060
)

print("\nSérie Temporal para 'TOTAL GERAL DO ATIVO' do Nubank:")
display(serie_nubank)

if not serie_nubank.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    serie_nubank.plot(ax=ax, marker='o', linestyle='-')
    ax.set_title("Evolução do Ativo Total - Nubank (COSIF Prudencial Doc 4060)", fontsize=16)
    ax.set_ylabel("Valor (em R$)")
    ax.set_xlabel("Data")
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.tight_layout()
    plt.show()

print("\n--- FIM DOS TESTES ---")

In [ ]:
# ==============================================================================
# CÉLULA: Gerar Tabela Comparativa a Partir de um Arquivo CSV (VERSÃO CORRIGIDA)
# ==============================================================================

# Passo 1: Carregar a lista de CNPJs do seu arquivo CSV
print("--- Lendo a lista de bancos do arquivo Banks.csv ---")
try:
    caminho_csv = Path('.') / 'Banks.csv'
    
    # ==========================================================================
    # CORREÇÃO CRÍTICA: Ler a coluna CNPJ como string para preservar os zeros.
    # ==========================================================================
    df_bancos_interesse = pd.read_csv(caminho_csv, dtype={'CNPJ': str}, sep=';')
    
    # Agora que a coluna já é string, podemos simplesmente convertê-la para uma lista.
    # Não há mais necessidade de .astype(str) aqui.
    lista_de_bancos_do_csv = df_bancos_interesse['CNPJ'].tolist()
    
    # Verificação adicional: Padronizar cada item da lista para 8 dígitos
    # Isso garante que mesmo que o CSV tenha CNPJs de 14 dígitos, eles serão tratados corretamente.
    # Usaremos a função que já temos no DataUtils para isso!
    lista_de_bancos_do_csv = [du.standardize_cnpj_base8(cnpj) for cnpj in lista_de_bancos_do_csv]
    
    print(f"Encontrados {len(lista_de_bancos_do_csv)} bancos para análise.")
    print("Amostra da lista (padronizada para 8 dígitos):", lista_de_bancos_do_csv[:5])

except FileNotFoundError:
    print(f"ERRO: O arquivo 'Banks.csv' não foi encontrado no diretório 'Analysis/'.")
    lista_de_bancos_do_csv = []

# Só continua se a lista de bancos não estiver vazia
if lista_de_bancos_do_csv:
    # Passo 2: Definir os indicadores (mesma lógica de antes)
    indicadores_desejados = {
        'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
        'TCB': {'tipo': 'Atributo', 'atributo': 'TCB_IFD_CAD'},
        'Volume de Operações tipo G': {'tipo': 'IFDATA', 'conta': 'G'},
        'Volume de Operações tipo H': {'tipo': 'IFDATA', 'conta': 'H'},
        'Total de Volume de Operações': {'tipo': 'IFDATA', 'conta': 'Total'},
        'Depósitos a Prazo': {'tipo': 'IFDATA', 'conta': 'Depósitos a Prazo (a4)'},
        'Letra de Crédito Imobiliário': {'tipo': 'IFDATA', 'conta': 'Letras de Crédito Imobiliário (c1)'},
        'Letra de Crédito Agronegócio': {'tipo': 'IFDATA', 'conta': 'Letras de Crédito do Agronegócio (c2)'},
        'Letras Financeiras': {'tipo': 'IFDATA', 'conta': 'Letras Financeiras (c3)'},
        'RWA': {'tipo': 'IFDATA', 'conta': 'Patrimônio de Referência para Comparação com o RWA'},
        'Ativo Total': {'tipo': 'IFDATA', 'conta': 'Ativo Total'}
        }

    # Passo 3: Chamar a função 'comparar_indicadores'
    print("\n--- Gerando tabela comparativa para os bancos do CSV ---")
    data_analise = 202412

    tabela_final = analisador.comparar_indicadores(
        identificadores=lista_de_bancos_do_csv,
        indicadores=indicadores_desejados,
        data=data_analise,
        fillna=0
    )

    # Passo 4: Exibir e salvar o resultado
    display(tabela_final)

    try:
        caminho_output_tabela = Path('.') / 'Results' / f'relatorio_bancos_{data_analise}.xlsx'
        tabela_final.to_excel(caminho_output_tabela)
        print(f"\nRelatório salvo com sucesso em: '{caminho_output_tabela.resolve()}'")
    except Exception as e:
        print(f"\nErro ao salvar o relatório em Excel: {e}")